In [1]:
# Install AgentChat and OpenAI client from Extensions
%pip install "autogen-ext[openai,azure,web-surfer]"

  Using cached pillow-11.1.0-cp312-cp312-macosx_10_13_x86_64.whl.metadata (9.1 kB)
Using cached pillow-11.1.0-cp312-cp312-macosx_10_13_x86_64.whl (3.2 MB)
  Attempting uninstall: pillow
    Found existing installation: pillow 10.4.0
    Uninstalling pillow-10.4.0:
      Successfully uninstalled pillow-10.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crawl4ai 0.4.248 requires pillow~=10.4, but you have pillow 11.1.0 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# %pip install crawl4ai # currently not compatible with autogen as it depends on a different version of pillow

%pip install ipywidgets


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load .env file
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

agent = AssistantAgent("assistant", AzureOpenAIChatCompletionClient(
    model="gpt-4o", 
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["OPENAI_API_VERSION"],
))
result = await agent.run(task="Say 'Hello World!'")
print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content="Say 'Hello World!'", type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=3), metadata={}, content='Hello World!', type='TextMessage')], stop_reason=None)


In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.agents.web_surfer import MultimodalWebSurfer

model_client = AzureOpenAIChatCompletionClient(
    model="gpt-4o", 
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["OPENAI_API_VERSION"],
)
assistant = AssistantAgent("assistant", model_client)
web_surfer = MultimodalWebSurfer("web_surfer", model_client)
user_proxy = UserProxyAgent("user_proxy")
termination = TextMentionTermination("exit") # Type 'exit' to end the conversation.
team = RoundRobinGroupChat([web_surfer, assistant, user_proxy], termination_condition=termination)
await Console(team.run_stream(task="Find information about AutoGen and write a short summary."))

---------- user ----------
Find information about AutoGen and write a short summary.
---------- web_surfer ----------
I typed 'AutoGen' into '0 個字元，共 2000 個字元'.

The web browser is open to the page [AutoGen - 搜尋](https://www.bing.com/search?q=AutoGen&form=QBLH&sp=-1&lq=0&pq=&sc=0-0&qs=n&sk=&cvid=5D9102702D5340AB9EBC2DACF920A462&ghsh=0&ghacc=0&ghpl=).
The viewport shows 25% of the webpage, and is positioned at the top of the page
The following text is visible in the viewport:

跳至內容
AutoGen搜尋優化
全部
Copilot
影片
圖片
地圖
新聞
更多
工具
約有 132,000 個結果microsoft.github.io
https://microsoft.github.io › autogen
AutoGen | AutoGen 0.2AutoGen provides multi-agent conversation framework as a high-level abstraction. With this framework, one can conveniently build LLM workflows. Easily Build Diverse Applications
BlogThis new package name was used to align with the new packages that will come …
DocsAutoGen aims to provide an easy-to-use and flexible framework for accelerating …
ExamplesExamples Automated Multi A

In [ ]:
await Console(team.run_stream(task="Summarize all news on HK01 in yesterday in cantonese"))

---------- user ----------
Summarize all news on HK01 in yesterday in cantonese
---------- web_surfer ----------
The webpage "最新即時新聞" from 香港01 provides a comprehensive overview of various current news categories, including local news, international affairs, lifestyle, and economics. A highlighted feature advises readers on coping with flight anxiety, suggesting three methods taught by doctors. Additionally, the site reports on a lunar surface observation expected to reveal an "X" shape, with expert recommendations on the best viewing times. Further coverage includes news about new drugs expected to enhance profitability for Hutchison's pharmaceutical division, and educational insights on helping children readjust to school routines. The page serves as a hub for the latest news and updates across different domains relevant to Hong Kong and international audiences.
---------- assistant ----------
以下是香港01昨天的主要新聞摘要：

1. **健康旅遊**: 提供旅遊指南，針對搭飛機時感到緊張的旅客，分享醫生建議的三招應對方法，以克服飛行恐懼。

2. **天文觀察**: 介

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Summarize all news on HK01 in yesterday in cantonese', type='TextMessage'), TextMessage(source='web_surfer', models_usage=RequestUsage(prompt_tokens=21334, completion_tokens=179), content='The webpage "最新即時新聞" from 香港01 provides a comprehensive overview of various current news categories, including local news, international affairs, lifestyle, and economics. A highlighted feature advises readers on coping with flight anxiety, suggesting three methods taught by doctors. Additionally, the site reports on a lunar surface observation expected to reveal an "X" shape, with expert recommendations on the best viewing times. Further coverage includes news about new drugs expected to enhance profitability for Hutchison\'s pharmaceutical division, and educational insights on helping children readjust to school routines. The page serves as a hub for the latest news and updates across different domains relevant to Hong Kong

In [2]:
await Console(team.run_stream(task="Summarize all news about GenAI on TechCrunch in yesterday in cantonese"))

NameError: name 'Console' is not defined

In [15]:
from crawl4ai import AsyncWebCrawler
from autogen_core.tools import FunctionTool
from autogen_core.tool_agent import ToolAgent

async def get_webpage_text(webpage_url: str):
    async with AsyncWebCrawler(verbose=True,headless=True) as crawler:
        result = await crawler.arun(
            url=webpage_url,
            bypass_cache=False,
            verbose=False,
        )
        return result.markdown

fetch_website = FunctionTool(get_webpage_text, description="Fetch website content to markdown from url.")

model_client = AzureOpenAIChatCompletionClient(
    model="gpt-4o", 
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["OPENAI_API_VERSION"],
)
assistant = AssistantAgent("assistant", model_client)
web_surfer = MultimodalWebSurfer("web_surfer", model_client)
user_proxy = UserProxyAgent("user_proxy")
fetch_website_agent = ToolAgent("Fetch a website content to be markdown from url", tool=fetch_website)
termination = TextMentionTermination("exit") # Type 'exit' to end the conversation.
team = RoundRobinGroupChat([web_surfer, fetch_website_agent, assistant, user_proxy], termination_condition=termination)

await Console(team.run_stream(task="Summarize all news about GenAI on TechCrunch in yesterday in cantonese"))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kan/my-workspaces/genai-samples/.venv/lib/python3.12/site-packages/autogen_ext/agents/web_surfer/page_script.js'